In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Lecture 1\n",
    "\n",
    "In this notebook, we first illustrate the basic machine learning workflow in Python (using the well-known [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)), and then show an implementation of the decision tree learning algorithm for classification.\n",
    "\n",
    "To run these examples, you need to have a Python installation that includes [scikit-learn](https://scikit-learn.org/stable/) (for machine learning), [matplotlib](https://matplotlib.org/) (for plotting), and [pandas](https://pandas.pydata.org/) (for loading the dataset). If you use the Anaconda distribution, you'll have these libraries out of the box.\n",
    "\n",
    "You'll also need to install [graphviz](https://pypi.org/project/graphviz/) to make the tree drawing work. If you use Anaconda, you need to install [graphviz](https://anaconda.org/anaconda/graphviz) and [python-graphviz](https://anaconda.org/conda-forge/python-graphviz)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "%config InlineBackend.figure_format = 'retina' \n",
    "plt.style.use('bmh')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Loading the iris data\n",
    "\n",
    "You can download the dataset as a CSV file [here](http://www.cse.chalmers.se/~richajo/dit866/lectures/l1/iris.csv). We first use [pandas](https://pandas.pydata.org/) to load the CSV file that stores the Iris data. Please note that you need to change the path to where you stored the csv file.\n",
    "\n",
    "We shuffle the dataset and split it into an input part `X` and an output part `Y`. In this case, we want to predict the type of iris: [*setosa*](https://en.wikipedia.org/wiki/Iris_setosa), [*versicolor*](https://en.wikipedia.org/wiki/Iris_versicolor), or [*virginica*](https://en.wikipedia.org/wiki/Iris_virginica)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "\n",
    "data = pd.read_csv('iris.csv')\n",
    "data_shuffled = data.sample(frac=1.0, random_state=0)\n",
    "X = data_shuffled.drop('species', axis=1)\n",
    "Y = data_shuffled['species']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's take a peek at the data. As you can see, there are four numerical column that we use as input, and then the discrete output column representing the species, which we'll use as the output."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>sepal_length</th>\n",
       "      <th>sepal_width</th>\n",
       "      <th>petal_length</th>\n",
       "      <th>petal_width</th>\n",
       "      <th>species</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>114</th>\n",
       "      <td>5.8</td>\n",
       "      <td>2.8</td>\n",
       "      <td>5.1</td>\n",
       "      <td>2.4</td>\n",
       "      <td>virginica</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>62</th>\n",
       "      <td>6.0</td>\n",
       "      <td>2.2</td>\n",
       "      <td>4.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>versicolor</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>33</th>\n",
       "      <td>5.5</td>\n",
       "      <td>4.2</td>\n",
       "      <td>1.4</td>\n",
       "      <td>0.2</td>\n",
       "      <td>setosa</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>107</th>\n",
       "      <td>7.3</td>\n",
       "      <td>2.9</td>\n",
       "      <td>6.3</td>\n",
       "      <td>1.8</td>\n",
       "      <td>virginica</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>5.0</td>\n",
       "      <td>3.4</td>\n",
       "      <td>1.5</td>\n",
       "      <td>0.2</td>\n",
       "      <td>setosa</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "     sepal_length  sepal_width  petal_length  petal_width     species\n",
       "114           5.8          2.8           5.1          2.4   virginica\n",
       "62            6.0          2.2           4.0          1.0  versicolor\n",
       "33            5.5          4.2           1.4          0.2      setosa\n",
       "107           7.3          2.9           6.3          1.8   virginica\n",
       "7             5.0          3.4           1.5          0.2      setosa"
      ]
     },
     "execution_count": 3,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data_shuffled.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The example above uses a pandas [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) to store the data. If you want to convert into a raw NumPy matrix, it can be done easily by calling `to_numpy`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[5.8, 2.8, 5.1, 2.4],\n",
       "       [6. , 2.2, 4. , 1. ],\n",
       "       [5.5, 4.2, 1.4, 0.2],\n",
       "       [7.3, 2.9, 6.3, 1.8],\n",
       "       [5. , 3.4, 1.5, 0.2]])"
      ]
     },
     "execution_count": 4,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "X.to_numpy()[:5]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Basic workflow\n",
    "\n",
    "Let's have some basic machine learning examples.\n",
    "\n",
    "We start by splitting the data into a training and test part. 40% of the data will be used for testing and the rest for training. We use the utility function `train_test_split` from the scikit-learn library.\n",
    "\n",
    "The train/test split is done randomly. The parameter `random_state` is used to set the random seed to a constant value, so that our results are reproducible."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import train_test_split\n",
    "\n",
    "Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.4, random_state=0)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We import a scikit-learn class representing [decision tree classifiers](https://en.wikipedia.org/wiki/Decision_tree_learning).\n",
    "\n",
    "We create a decision tree classifier and train it on the training set (`fit`)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.tree import DecisionTreeClassifier\n",
    "\n",
    "clf = DecisionTreeClassifier()\n",
    "clf.fit(Xtrain, Ytrain);"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's see what the classifier says about one particular instance. We take a look at the first instance in the test set:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>sepal_length</th>\n",
       "      <th>sepal_width</th>\n",
       "      <th>petal_length</th>\n",
       "      <th>petal_width</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>118</th>\n",
       "      <td>7.7</td>\n",
       "      <td>2.6</td>\n",
       "      <td>6.9</td>\n",
       "      <td>2.3</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "     sepal_length  sepal_width  petal_length  petal_width\n",
       "118           7.7          2.6           6.9          2.3"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "Xtest.head(1)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We comput the classifier's prediction for this instance, and in this case the classifier thinks that this is a [*virginica*](https://en.wikipedia.org/wiki/Iris_virginica)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['virginica'], dtype=object)"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "one_instance = Xtest.head(1)\n",
    "\n",
    "clf.predict(one_instance)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "By looking at the output part of the test set, we can verify that the classifier was correct in this case."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "118    virginica\n",
       "Name: species, dtype: object"
      ]
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "Ytest.head(1)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We can compute predictions for all instances in the test set. Note that the `predict` method for scikit-learn predictors normally expects a *collection* of instances, not a single instance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['virginica', 'virginica', 'virginica', 'setosa', 'versicolor',\n",
       "       'setosa', 'virginica', 'virginica', 'virginica', 'versicolor',\n",
       "       'setosa', 'virginica', 'setosa', 'virginica', 'versicolor',\n",
       "       'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor',\n",
       "       'virginica', 'versicolor', 'virginica', 'versicolor', 'virginica',\n",
       "       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'virginica',\n",
       "       'virginica', 'versicolor', 'versicolor', 'versicolor', 'virginica',\n",
       "       'setosa', 'virginica', 'setosa', 'virginica', 'versicolor',\n",
       "       'setosa', 'virginica', 'setosa', 'virginica', 'versicolor',\n",
       "       'virginica', 'setosa', 'virginica', 'versicolor', 'setosa',\n",
       "       'versicolor', 'versicolor', 'virginica', 'setosa', 'virginica',\n",
       "       'setosa', 'virginica', 'versicolor', 'versicolor'], dtype=object)"
      ]
     },
     "execution_count": 10,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "all_predictions = clf.predict(Xtest)\n",
    "\n",
    "all_predictions"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "How good are our predictions? We compute the *accuracy* of our classifier for this test set. The accuracy is defined as the proportion of right answers. (See [here](https://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers) for some other ways to evaluate classifiers.)\n",
    "\n",
    "The iris dataset is quite easy, and the accuracy is quite high for our classifier."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9333333333333333"
      ]
     },
     "execution_count": 11,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.metrics import accuracy_score\n",
    "\n",
    "accuracy_score(Ytest, all_predictions)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Alternatively, we can call the method `score` for our classifier. This method will predict on the test set and then evaluate the predictions using the accuracy."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9333333333333333"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "clf.score(Xtest, Ytest)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Normally, we'll carry out several evaluations while we are selecting the best model. This will be done using a separate *validation set*, like a second test set, or using [*cross-validation*](https://en.wikipedia.org/wiki/Cross-validation_(statistics)).\n",
    "\n",
    "In scikit-learn, there are a couple of ways that we can do cross-validation. The simplest way is to call `cross_val_score`, which for each cross-validation fold will call the method `score` mentioned above."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([1.        , 0.94444444, 0.94444444, 1.        , 1.        ])"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.model_selection import cross_val_score\n",
    "\n",
    "cross_val_score(clf, Xtrain, Ytrain, cv=5)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Alternatively, you can call `cross_validate`, where you can specify what kind of metric to use. The output of this function is also a bit more detailed."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{'fit_time': array([0.00246549, 0.00229478, 0.00251865, 0.00256848, 0.00195932]),\n",
       " 'score_time': array([0.00174689, 0.00270343, 0.00230718, 0.0013659 , 0.00151634]),\n",
       " 'test_score': array([1.        , 0.94444444, 0.94444444, 1.        , 1.        ])}"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.model_selection import cross_validate\n",
    "\n",
    "cross_validate(clf, Xtrain, Ytrain, cv=5, scoring='accuracy')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Finally, let's evaluate some other classifiers in addition to the decision trees. We first consider linear [support vector classifiers](https://en.wikipedia.org/wiki/Support-vector_machine)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{'fit_time': array([0.01014996, 0.01492524, 0.00867295, 0.00882149, 0.00952196]),\n",
       " 'score_time': array([0.03162599, 0.00138974, 0.00126338, 0.00127745, 0.00170469]),\n",
       " 'test_score': array([1.        , 0.94444444, 0.94444444, 0.94444444, 1.        ])}"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.svm import LinearSVC\n",
    "\n",
    "clf2 = LinearSVC(max_iter=10000)\n",
    "cross_validate(clf2, Xtrain, Ytrain, cv=5, scoring='accuracy')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "It is useful to compare classifiers to a *baseline*, such as a trivial majority-class classifier."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{'fit_time': array([0.00077176, 0.00155926, 0.00070357, 0.00092196, 0.00110078]),\n",
       " 'score_time': array([0.00082517, 0.00102472, 0.00063992, 0.00106192, 0.00061417]),\n",
       " 'test_score': array([0.33333333, 0.33333333, 0.38888889, 0.38888889, 0.33333333])}"
      ]
     },
     "execution_count": 16,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.dummy import DummyClassifier\n",
    "\n",
    "majority_baseline = DummyClassifier(strategy='most_frequent')\n",
    "cross_validate(majority_baseline, Xtrain, Ytrain, cv=5, scoring='accuracy')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Illustrating the iris example\n",
    "\n",
    "We'll provide some illustrations of how the various types of classifiers work by drawing the *decision boundaries* for a two-dimensional dataset.\n",
    "\n",
    "Since the iris dataset has four dimensions, we select two columns (petal length and width) so that we can plot the data in a two-dimensional scatterplot."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/tmp/ipykernel_1295525/3779132730.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`\n",
      "  Y_encoded = Y.replace({'setosa':0, 'versicolor':1, 'virginica':2})\n"
     ]
    },
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 500x500 with 1 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 423,
       "width": 434
      }
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "X2 = X[['petal_length', 'petal_width']]\n",
    "Y_encoded = Y.replace({'setosa':0, 'versicolor':1, 'virginica':2})\n",
    "\n",
    "plt.figure(figsize=(5,5))\n",
    "plt.scatter(X2.petal_length, X2.petal_width, c=Y_encoded, cmap='tab10');"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We define a utility function that makes a scatterplot of a dataset and also draws the decision boundary. Here, we're using a bit of NumPy and matplotlib tricks. Don't worry about the details."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_boundary(clf, X, Y, cmap='tab10', names=None):\n",
    "\n",
    "    if isinstance(X, pd.DataFrame):\n",
    "        if not names:\n",
    "            names = list(X.columns)\n",
    "        X = X.to_numpy()\n",
    "\n",
    "    x_min, x_max = X[:,0].min(), X[:,0].max()\n",
    "    y_min, y_max = X[:,1].min(), X[:,1].max()\n",
    "\n",
    "    x_off = (x_max-x_min)/25\n",
    "    y_off = (y_max-y_min)/25\n",
    "    x_min -= x_off\n",
    "    x_max += x_off\n",
    "    y_min -= y_off\n",
    "    y_max += y_off\n",
    "    \n",
    "    xs = np.linspace(x_min, x_max, 250)\n",
    "    ys = np.linspace(y_min, y_max, 250)\n",
    "    \n",
    "    xx, yy = np.meshgrid(xs, ys)\n",
    "    \n",
    "    lenc = {c:i for i, c in enumerate(clf.classes_)}\n",
    "    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])\n",
    "    Z = np.array([lenc[z] for z in Z])\n",
    "    Z = Z.reshape(xx.shape)\n",
    "    Yenc = [lenc[y] for y in Y]\n",
    "    plt.figure(figsize=(5,5))\n",
    "    plt.contourf(xx, yy, Z, cmap=cmap, alpha=0.15)\n",
    "    plt.contour(xx, yy, Z, colors='k', linewidths=0.2)\n",
    "        \n",
    "    sc = plt.scatter(X[:,0], X[:,1], c=Yenc, cmap=cmap, alpha=0.5, edgecolors='k', linewidths=0.5);\n",
    "    \n",
    "    plt.legend(handles=sc.legend_elements()[0], labels=list(clf.classes_))\n",
    "    \n",
    "    if names:\n",
    "        plt.xlabel(names[0])\n",
    "        plt.ylabel(names[1])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now we can plot some decision boundaries. Here, we see the how it looks for a linear classifier:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/home/richard/external-tools/miniconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.\n",
      "  warnings.warn(\n",
      "/home/richard/external-tools/miniconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names\n",
      "  warnings.warn(\n"
     ]
    },
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 500x500 with 1 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 446,
       "width": 456
      }
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "from sklearn.svm import LinearSVC\n",
    "\n",
    "cls = LinearSVC()\n",
    "cls.fit(X2, Y)\n",
    "plot_boundary(cls, X2, Y)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "And for a decision tree, the decision boundary has the characteristic shape with right angles."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/home/richard/external-tools/miniconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names\n",
      "  warnings.warn(\n"
     ]
    },
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 500x500 with 1 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 446,
       "width": 456
      }
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "cls = DecisionTreeClassifier()\n",
    "cls.fit(X2, Y)\n",
    "plot_boundary(cls, X2, Y)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Decision tree implementation\n",
    "\n",
    "In the following section, we'll implement a learning algorithm for decision tree classifiers.\n",
    "\n",
    "To represent a decision tree, we define the two components of decision trees: leaves and branches. A *leaf* corresponds to the base case of a recursion. It is a \"dummy\" tree that always returns a constant value.\n",
    "\n",
    "The code that is used to draw the trees can be ignored if you want."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "class DecisionTreeLeaf:\n",
    "\n",
    "    def __init__(self, value):\n",
    "        self.value = value\n",
    "\n",
    "    # This method computes the prediction for this leaf node. This will just return a constant value.\n",
    "    def predict(self, x):\n",
    "        return self.value\n",
    "\n",
    "    # Utility function to draw a tree visually using graphviz.\n",
    "    def draw_tree(self, graph, node_counter, names):\n",
    "        node_id = str(node_counter)\n",
    "        val_str = f'{self.value:.4g}' if isinstance(self.value, float) else str(self.value)\n",
    "        graph.node(node_id, val_str, style='filled')\n",
    "        return node_counter+1, node_id\n",
    "        \n",
    "    def __eq__(self, other):\n",
    "        if isinstance(other, DecisionTreeLeaf):\n",
    "            return self.value == other.value\n",
    "        else:\n",
    "            return False\n",
    "        \n",
    "    def to_str(self, indent, lines, names):\n",
    "        val_str = f'{self.value:.4g}' if isinstance(self.value, float) else str(self.value)\n",
    "        lines.append(' '*indent + val_str)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A *branch* will look at one feature, and will select a subtree depending on the value of the feature. That subtree will then be called recursively to compute the prediction.\n",
    "\n",
    "This implementation assumes that the feature is numerical. Depending on whether the feature is or isn't greater than a threshold, the \"high\" or \"low\" subtree will be selected."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [],
   "source": [
    "class DecisionTreeBranch:\n",
    "\n",
    "    def __init__(self, feature, threshold, low_subtree, high_subtree):\n",
    "        self.feature = feature\n",
    "        self.threshold = threshold\n",
    "        self.low_subtree = low_subtree\n",
    "        self.high_subtree = high_subtree\n",
    "\n",
    "    # For a branch node, we compute the prediction by first considering the feature, and then \n",
    "    # calling the upper or lower subtree, depending on whether the feature is or isn't greater\n",
    "    # than the threshold.\n",
    "    def predict(self, x):\n",
    "        if x[self.feature] <= self.threshold:\n",
    "            return self.low_subtree.predict(x)\n",
    "        else:\n",
    "            return self.high_subtree.predict(x)\n",
    "\n",
    "    # Utility function to draw a tree visually using graphviz.\n",
    "    def draw_tree(self, graph, node_counter, names):\n",
    "        node_counter, low_id = self.low_subtree.draw_tree(graph, node_counter, names)\n",
    "        node_counter, high_id = self.high_subtree.draw_tree(graph, node_counter, names)\n",
    "        node_id = str(node_counter)\n",
    "        fname = f'F{self.feature}' if names is None else names[self.feature]\n",
    "        lbl = f'{fname} > {self.threshold:.4g}?'\n",
    "        graph.node(node_id, lbl, shape='box', fillcolor='yellow', style='filled, rounded')\n",
    "        graph.edge(node_id, low_id, 'False')\n",
    "        graph.edge(node_id, high_id, 'True')\n",
    "        return node_counter+1, node_id\n",
    "        \n",
    "    def to_str(self, indent, lines, names):\n",
    "        pad = ' '*indent\n",
    "        fname = f'F{self.feature}' if names is None else names[self.feature]\n",
    "        lines.append(pad + f'{fname} <= {self.threshold:.3g}?')\n",
    "        lines.append(pad + '  True =>')\n",
    "        self.low_subtree.to_str(indent+4, lines, names)\n",
    "        lines.append(pad + '  False =>')\n",
    "        self.high_subtree.to_str(indent+4, lines, names)\n",
    "        "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now, we have the components needed to implement decision tree classifiers and regression models.\n",
    "\n",
    "Following standard practice in scikit-learn, we inherit from the class [`BaseEstimator`](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html), which is the base class of all classifiers and regression models.\n",
    "\n",
    "We write the `DecisionTree` class as an abstract class that contains the functionality common to all types of decision trees. Classification and regression models will be implemented as subclasses. The classification subclass is given below, while the regression subclass will be implemented as a part of the first assignment. Following scikit-learn naming conventions, we'll call the training and prediction methods `fit` and `predict`, respectively.\n",
    "\n",
    "The functions that the subclasses will need to implement, and which are done differently for classification and regression, are the following:\n",
    "\n",
    "* `get_default_value`: what output value to use if we decide to return a leaf node. For classification, this will be the most common output value, while for regression it will be the mean.\n",
    "* `is_homogeneous`: tests whether a set of output values is homogeneous. For classification, this means that all outputs are identical; for regression, we'll probably test whether the variance is smaller than some threshold.\n",
    "* `best_split`: finds the best splitting point for that feature. For classification, this will be based on one of the classification criteria (information gain, gini impurity, majority sum); for regression, it will be based on variances in the subsets."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [],
   "source": [
    "from graphviz import Digraph\n",
    "from sklearn.base import BaseEstimator, ClassifierMixin\n",
    "from abc import ABC, abstractmethod\n",
    "\n",
    "class DecisionTree(ABC, BaseEstimator):\n",
    "\n",
    "    def __init__(self, max_depth):\n",
    "        super().__init__()\n",
    "        self.max_depth = max_depth\n",
    "        \n",
    "    # As usual in scikit-learn, the training method is called *fit*. We first process the dataset so that\n",
    "    # we're sure that it's represented as a NumPy matrix. Then we call the recursive tree-building method\n",
    "    # called make_tree (see below).\n",
    "    def fit(self, X, Y):\n",
    "        if isinstance(X, pd.DataFrame):\n",
    "            self.names = X.columns\n",
    "            X = X.to_numpy()\n",
    "        elif isinstance(X, list):\n",
    "            self.names = None\n",
    "            X = np.array(X)\n",
    "        else:\n",
    "            self.names = None\n",
    "        Y = np.array(Y)        \n",
    "        self.root = self.make_tree(X, Y, self.max_depth)\n",
    "        \n",
    "    def draw_tree(self):\n",
    "        graph = Digraph()\n",
    "        self.root.draw_tree(graph, 0, self.names)\n",
    "        return graph\n",
    "    \n",
    "    # By scikit-learn convention, the method *predict* computes the classification or regression output\n",
    "    # for a set of instances.\n",
    "    # To implement it, we call a separate method that carries out the prediction for one instance.\n",
    "    def predict(self, X):\n",
    "        if isinstance(X, pd.DataFrame):\n",
    "            X = X.to_numpy()\n",
    "        return [self.predict_one(x) for x in X]\n",
    "\n",
    "    # Predicting the output for one instance.\n",
    "    def predict_one(self, x):\n",
    "        return self.root.predict(x)        \n",
    "\n",
    "    # This is the recursive training \n",
    "    def make_tree(self, X, Y, max_depth):\n",
    "\n",
    "        # We start by computing the default value that will be used if we'll return a leaf node.\n",
    "        # For classifiers, this will be the most common value in Y.\n",
    "        default_value = self.get_default_value(Y)\n",
    "\n",
    "        # First the two base cases in the recursion: is the training set completely\n",
    "        # homogeneous, or have we reached the maximum depth? Then we need to return a leaf.\n",
    "\n",
    "        # If we have reached the maximum depth, return a leaf with the majority value.\n",
    "        if max_depth == 0:\n",
    "            return DecisionTreeLeaf(default_value)\n",
    "\n",
    "        # If all the instances in the remaining training set have the same output value,\n",
    "        # return a leaf with this value.\n",
    "        if self.is_homogeneous(Y):\n",
    "            return DecisionTreeLeaf(default_value)\n",
    "\n",
    "        # Select the \"most useful\" feature and split threshold. To rank the \"usefulness\" of features,\n",
    "        # we use one of the classification or regression criteria.\n",
    "        # For each feature, we call best_split (defined in a subclass). We then maximize over the features.\n",
    "        n_features = X.shape[1]\n",
    "        \n",
    "        _, best_feature, best_threshold = \\\n",
    "            max(self.best_split(X, Y, feature) \n",
    "                for feature in range(n_features))\n",
    "        \n",
    "        if best_feature is None:\n",
    "            return DecisionTreeLeaf(default_value)\n",
    "\n",
    "        # Split the training set into subgroups, based on whether the selected feature is greater than\n",
    "        # the threshold or not\n",
    "        X_low, X_high, Y_low, Y_high = \\\n",
    "            self.split_by_feature(X, Y, best_feature, best_threshold)\n",
    "\n",
    "        # Build the subtrees using a recursive call. Each subtree is associated\n",
    "        # with a value of the feature.\n",
    "        low_subtree = self.make_tree(X_low, Y_low, max_depth-1)\n",
    "        high_subtree = self.make_tree(X_high, Y_high, max_depth-1)\n",
    "\n",
    "        if low_subtree == high_subtree:\n",
    "            return low_subtree\n",
    "\n",
    "        # Return a decision tree branch containing the result.\n",
    "        return DecisionTreeBranch(best_feature, best_threshold, low_subtree, high_subtree)\n",
    "    \n",
    "    # Utility method that splits the data into the \"upper\" and \"lower\" part, based on a feature\n",
    "    # and a threshold.\n",
    "    def split_by_feature(self, X, Y, feature, threshold):\n",
    "        low = X[:,feature] <= threshold\n",
    "        high = ~low\n",
    "        return X[low], X[high], Y[low], Y[high]\n",
    "    \n",
    "    def __str__(self):\n",
    "        lines = []\n",
    "        self.root.to_str(0, lines, self.names)\n",
    "        return '\\n'.join(lines)\n",
    "    \n",
    "    # The following three methods need to be implemented by the classification and regression subclasses.\n",
    "    \n",
    "    @abstractmethod\n",
    "    def get_default_value(self, Y):\n",
    "        pass\n",
    "\n",
    "    @abstractmethod\n",
    "    def is_homogeneous(self, Y):\n",
    "        pass\n",
    "\n",
    "    @abstractmethod\n",
    "    def best_split(self, X, Y, feature):\n",
    "        pass"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This is the subclass that implements decision tree classification. This implementation makes heavy use of the [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) class, which is a standard Python data structure for frequency counting."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [],
   "source": [
    "from collections import Counter\n",
    "\n",
    "class TreeClassifier(DecisionTree, ClassifierMixin):\n",
    "\n",
    "    def __init__(self, max_depth=10, criterion='maj_sum'):\n",
    "        super().__init__(max_depth)\n",
    "        self.criterion = criterion\n",
    "        \n",
    "    def fit(self, X, Y):\n",
    "        # For decision tree classifiers, there are some different ways to measure\n",
    "        # the homogeneity of subsets.\n",
    "        if self.criterion == 'maj_sum':\n",
    "            self.criterion_function = majority_sum_scorer\n",
    "        elif self.criterion == 'info_gain':\n",
    "            self.criterion_function = info_gain_scorer\n",
    "        elif self.criterion == 'gini':\n",
    "            self.criterion_function = gini_scorer\n",
    "        else:\n",
    "            raise Exception(f'Unknown criterion: {self.criterion}')\n",
    "        super().fit(X, Y)\n",
    "        self.classes_ = sorted(set(Y))\n",
    "\n",
    "    # Select a default value that is going to be used if we decide to make a leaf.\n",
    "    # We will select the most common value.\n",
    "    def get_default_value(self, Y):\n",
    "        self.class_distribution = Counter(Y)\n",
    "        return self.class_distribution.most_common(1)[0][0]\n",
    "    \n",
    "    # Checks whether a set of output values is homogeneous. In the classification case, \n",
    "    # this means that all output values are identical.\n",
    "    # We assume that we called get_default_value just before, so that we can access\n",
    "    # the class_distribution attribute. If the class distribution contains just one item,\n",
    "    # this means that the set is homogeneous.\n",
    "    def is_homogeneous(self, Y):\n",
    "        return len(self.class_distribution) == 1\n",
    "        \n",
    "    # Finds the best splitting point for a given feature. We'll keep frequency tables (Counters)\n",
    "    # for the upper and lower parts, and then compute the impurity criterion using these tables.\n",
    "    # In the end, we return a triple consisting of\n",
    "    # - the best score we found, according to the criterion we're using\n",
    "    # - the id of the feature\n",
    "    # - the threshold for the best split\n",
    "    def best_split(self, X, Y, feature):\n",
    "\n",
    "        # Create a list of input-output pairs, where we have sorted\n",
    "        # in ascending order by the input feature we're considering.\n",
    "        sorted_indices = np.argsort(X[:, feature])        \n",
    "        X_sorted = list(X[sorted_indices, feature])\n",
    "        Y_sorted = list(Y[sorted_indices])\n",
    "\n",
    "        n = len(Y)\n",
    "\n",
    "        # The frequency tables corresponding to the parts *before and including*\n",
    "        # and *after* the current element.\n",
    "        low_distr = Counter()\n",
    "        high_distr = Counter(Y)\n",
    "\n",
    "        # Keep track of the best result we've seen so far.\n",
    "        max_score = -np.inf\n",
    "        max_i = None\n",
    "\n",
    "        # Go through all the positions (excluding the last position).\n",
    "        for i in range(0, n-1):\n",
    "\n",
    "            # Input and output at the current position.\n",
    "            x_i = X_sorted[i]\n",
    "            y_i = Y_sorted[i]\n",
    "            \n",
    "            # Update the frequency tables.\n",
    "            low_distr[y_i] += 1\n",
    "            high_distr[y_i] -= 1\n",
    "\n",
    "            # If the input is equal to the input at the next position, we will\n",
    "            # not consider a split here.\n",
    "            #x_next = XY[i+1][0]\n",
    "            x_next = X_sorted[i+1]\n",
    "            if x_i == x_next:\n",
    "                continue\n",
    "\n",
    "            # Compute the homogeneity criterion for a split at this position.\n",
    "            score = self.criterion_function(i+1, low_distr, n-i-1, high_distr)\n",
    "\n",
    "            # If this is the best split, remember it.\n",
    "            if score > max_score:\n",
    "                max_score = score\n",
    "                max_i = i\n",
    "\n",
    "        # If we didn't find any split (meaning that all inputs are identical), return\n",
    "        # a dummy value.\n",
    "        if max_i is None:\n",
    "            return -np.inf, None, None\n",
    "\n",
    "        # Otherwise, return the best split we found and its score.\n",
    "        split_point = 0.5*(X_sorted[max_i] + X_sorted[max_i+1])\n",
    "        return max_score, feature, split_point\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Here, we define the various criteria we can use to find the \"quality\" of a split in terms of how homogeneous the subsets are. See the reading material for the mathematical definitions of these criteria."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [],
   "source": [
    "def majority_sum_scorer(n_low, low_distr, n_high, high_distr):\n",
    "    maj_sum_low = low_distr.most_common(1)[0][1]\n",
    "    maj_sum_high = high_distr.most_common(1)[0][1]\n",
    "    return maj_sum_low + maj_sum_high\n",
    "    \n",
    "def entropy(distr):\n",
    "    n = sum(distr.values())\n",
    "    ps = [n_i/n for n_i in distr.values()]\n",
    "    return -sum(p*np.log2(p) if p > 0 else 0 for p in ps)\n",
    "\n",
    "def info_gain_scorer(n_low, low_distr, n_high, high_distr):\n",
    "    return -(n_low*entropy(low_distr)+n_high*entropy(high_distr))/(n_low+n_high)\n",
    "\n",
    "def gini_impurity(distr):\n",
    "    n = sum(distr.values())\n",
    "    ps = [n_i/n for n_i in distr.values()]\n",
    "    return 1-sum(p**2 for p in ps)\n",
    "    \n",
    "def gini_scorer(n_low, low_distr, n_high, high_distr):\n",
    "    return -(n_low*gini_impurity(low_distr)+n_high*gini_impurity(high_distr))/(n_low+n_high)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Iris example, continued"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now, let's train decision trees for the iris dataset, using our own implementation (and not the one that's included in scikit-learn, which we used previously).\n",
    "\n",
    "We can investigate how the `max_depth` hyperparameter affects the decision boundary. Generally, if we allow deeper trees, we can express more complex boundaries. This allows us to handle more \"difficult\" datasets but also introduces the risk of overfitting.\n",
    "\n",
    "(If you didn't install graphviz, you can just remove the line that includes `draw_tree`.)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"UTF-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       " \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Generated by graphviz version 2.50.0 (0)\n",
       " -->\n",
       "<!-- Pages: 1 -->\n",
       "<svg width=\"249pt\" height=\"218pt\"\n",
       " viewBox=\"0.00 0.00 248.74 218.00\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       "<g id=\"graph0\" class=\"graph\" transform=\"scale(1 1) rotate(0) translate(4 214)\">\n",
       "<polygon fill=\"white\" stroke=\"transparent\" points=\"-4,4 -4,-214 244.74,-214 244.74,4 -4,4\"/>\n",
       "<!-- 0 -->\n",
       "<g id=\"node1\" class=\"node\">\n",
       "<title>0</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"32.5\" cy=\"-105\" rx=\"32.49\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"32.5\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\">setosa</text>\n",
       "</g>\n",
       "<!-- 1 -->\n",
       "<g id=\"node2\" class=\"node\">\n",
       "<title>1</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"92.5\" cy=\"-18\" rx=\"46.29\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"92.5\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\">versicolor</text>\n",
       "</g>\n",
       "<!-- 2 -->\n",
       "<g id=\"node3\" class=\"node\">\n",
       "<title>2</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"198.5\" cy=\"-18\" rx=\"42.49\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"198.5\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\">virginica</text>\n",
       "</g>\n",
       "<!-- 3 -->\n",
       "<g id=\"node4\" class=\"node\">\n",
       "<title>3</title>\n",
       "<path fill=\"yellow\" stroke=\"black\" d=\"M196.5,-123C196.5,-123 94.5,-123 94.5,-123 88.5,-123 82.5,-117 82.5,-111 82.5,-111 82.5,-99 82.5,-99 82.5,-93 88.5,-87 94.5,-87 94.5,-87 196.5,-87 196.5,-87 202.5,-87 208.5,-93 208.5,-99 208.5,-99 208.5,-111 208.5,-111 208.5,-117 202.5,-123 196.5,-123\"/>\n",
       "<text text-anchor=\"middle\" x=\"145.5\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\">petal_width &gt; 1.65?</text>\n",
       "</g>\n",
       "<!-- 3&#45;&gt;1 -->\n",
       "<g id=\"edge1\" class=\"edge\">\n",
       "<title>3&#45;&gt;1</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M134.77,-86.8C127.13,-74.54 116.74,-57.87 108.17,-44.13\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"111.11,-42.23 102.85,-35.6 105.17,-45.93 111.11,-42.23\"/>\n",
       "<text text-anchor=\"middle\" x=\"136\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\">False</text>\n",
       "</g>\n",
       "<!-- 3&#45;&gt;2 -->\n",
       "<g id=\"edge2\" class=\"edge\">\n",
       "<title>3&#45;&gt;2</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M156.22,-86.8C163.86,-74.54 174.26,-57.87 182.83,-44.13\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"185.83,-45.93 188.15,-35.6 179.89,-42.23 185.83,-45.93\"/>\n",
       "<text text-anchor=\"middle\" x=\"187.5\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\">True</text>\n",
       "</g>\n",
       "<!-- 4 -->\n",
       "<g id=\"node5\" class=\"node\">\n",
       "<title>4</title>\n",
       "<path fill=\"yellow\" stroke=\"black\" d=\"M121,-210C121,-210 26,-210 26,-210 20,-210 14,-204 14,-198 14,-198 14,-186 14,-186 14,-180 20,-174 26,-174 26,-174 121,-174 121,-174 127,-174 133,-180 133,-186 133,-186 133,-198 133,-198 133,-204 127,-210 121,-210\"/>\n",
       "<text text-anchor=\"middle\" x=\"73.5\" y=\"-188.3\" font-family=\"Times,serif\" font-size=\"14.00\">petal_width &gt; 0.8?</text>\n",
       "</g>\n",
       "<!-- 4&#45;&gt;0 -->\n",
       "<g id=\"edge3\" class=\"edge\">\n",
       "<title>4&#45;&gt;0</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M65.2,-173.8C59.4,-161.78 51.55,-145.51 45,-131.92\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"48,-130.08 40.5,-122.6 41.69,-133.12 48,-130.08\"/>\n",
       "<text text-anchor=\"middle\" x=\"70\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\">False</text>\n",
       "</g>\n",
       "<!-- 4&#45;&gt;3 -->\n",
       "<g id=\"edge4\" class=\"edge\">\n",
       "<title>4&#45;&gt;3</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M88.07,-173.8C98.51,-161.47 112.73,-144.68 124.41,-130.89\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"127.16,-133.07 130.95,-123.18 121.81,-128.54 127.16,-133.07\"/>\n",
       "<text text-anchor=\"middle\" x=\"126.5\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\">True</text>\n",
       "</g>\n",
       "</g>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<graphviz.graphs.Digraph at 0x7f3354d1b750>"
      ]
     },
     "execution_count": 26,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cls = TreeClassifier(max_depth=2)\n",
    "cls.fit(X2, Y)\n",
    "cls.draw_tree()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "petal_width <= 0.8?\n",
      "  True =>\n",
      "    setosa\n",
      "  False =>\n",
      "    petal_width <= 1.65?\n",
      "      True =>\n",
      "        versicolor\n",
      "      False =>\n",
      "        virginica\n"
     ]
    }
   ],
   "source": [
    "print(cls)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 500x500 with 1 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 446,
       "width": 456
      }
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "plot_boundary(cls, X2, Y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"UTF-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       " \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Generated by graphviz version 2.50.0 (0)\n",
       " -->\n",
       "<!-- Pages: 1 -->\n",
       "<svg width=\"492pt\" height=\"479pt\"\n",
       " viewBox=\"0.00 0.00 492.39 479.00\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       "<g id=\"graph0\" class=\"graph\" transform=\"scale(1 1) rotate(0) translate(4 475)\">\n",
       "<polygon fill=\"white\" stroke=\"transparent\" points=\"-4,4 -4,-475 488.39,-475 488.39,4 -4,4\"/>\n",
       "<!-- 0 -->\n",
       "<g id=\"node1\" class=\"node\">\n",
       "<title>0</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"181.15\" cy=\"-366\" rx=\"32.49\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"181.15\" y=\"-362.3\" font-family=\"Times,serif\" font-size=\"14.00\">setosa</text>\n",
       "</g>\n",
       "<!-- 1 -->\n",
       "<g id=\"node2\" class=\"node\">\n",
       "<title>1</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"46.15\" cy=\"-105\" rx=\"46.29\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"46.15\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\">versicolor</text>\n",
       "</g>\n",
       "<!-- 2 -->\n",
       "<g id=\"node3\" class=\"node\">\n",
       "<title>2</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"152.15\" cy=\"-105\" rx=\"42.49\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"152.15\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\">virginica</text>\n",
       "</g>\n",
       "<!-- 3 -->\n",
       "<g id=\"node4\" class=\"node\">\n",
       "<title>3</title>\n",
       "<path fill=\"yellow\" stroke=\"black\" d=\"M192.15,-210C192.15,-210 90.15,-210 90.15,-210 84.15,-210 78.15,-204 78.15,-198 78.15,-198 78.15,-186 78.15,-186 78.15,-180 84.15,-174 90.15,-174 90.15,-174 192.15,-174 192.15,-174 198.15,-174 204.15,-180 204.15,-186 204.15,-186 204.15,-198 204.15,-198 204.15,-204 198.15,-210 192.15,-210\"/>\n",
       "<text text-anchor=\"middle\" x=\"141.15\" y=\"-188.3\" font-family=\"Times,serif\" font-size=\"14.00\">petal_width &gt; 1.65?</text>\n",
       "</g>\n",
       "<!-- 3&#45;&gt;1 -->\n",
       "<g id=\"edge1\" class=\"edge\">\n",
       "<title>3&#45;&gt;1</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M121.92,-173.8C107.42,-160.83 87.4,-142.91 71.58,-128.76\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"73.53,-125.8 63.74,-121.74 68.86,-131.02 73.53,-125.8\"/>\n",
       "<text text-anchor=\"middle\" x=\"114.65\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\">False</text>\n",
       "</g>\n",
       "<!-- 3&#45;&gt;2 -->\n",
       "<g id=\"edge2\" class=\"edge\">\n",
       "<title>3&#45;&gt;2</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M143.37,-173.8C144.88,-162.16 146.9,-146.55 148.62,-133.24\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"152.11,-133.54 149.92,-123.18 145.17,-132.64 152.11,-133.54\"/>\n",
       "<text text-anchor=\"middle\" x=\"161.15\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\">True</text>\n",
       "</g>\n",
       "<!-- 4 -->\n",
       "<g id=\"node5\" class=\"node\">\n",
       "<title>4</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"264.15\" cy=\"-105\" rx=\"42.49\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"264.15\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\">virginica</text>\n",
       "</g>\n",
       "<!-- 5 -->\n",
       "<g id=\"node6\" class=\"node\">\n",
       "<title>5</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"336.15\" cy=\"-18\" rx=\"46.29\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"336.15\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\">versicolor</text>\n",
       "</g>\n",
       "<!-- 6 -->\n",
       "<g id=\"node7\" class=\"node\">\n",
       "<title>6</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"442.15\" cy=\"-18\" rx=\"42.49\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"442.15\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\">virginica</text>\n",
       "</g>\n",
       "<!-- 7 -->\n",
       "<g id=\"node8\" class=\"node\">\n",
       "<title>7</title>\n",
       "<path fill=\"yellow\" stroke=\"black\" d=\"M441.65,-123C441.65,-123 336.65,-123 336.65,-123 330.65,-123 324.65,-117 324.65,-111 324.65,-111 324.65,-99 324.65,-99 324.65,-93 330.65,-87 336.65,-87 336.65,-87 441.65,-87 441.65,-87 447.65,-87 453.65,-93 453.65,-99 453.65,-99 453.65,-111 453.65,-111 453.65,-117 447.65,-123 441.65,-123\"/>\n",
       "<text text-anchor=\"middle\" x=\"389.15\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\">petal_length &gt; 5.45?</text>\n",
       "</g>\n",
       "<!-- 7&#45;&gt;5 -->\n",
       "<g id=\"edge3\" class=\"edge\">\n",
       "<title>7&#45;&gt;5</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M378.42,-86.8C370.78,-74.54 360.38,-57.87 351.81,-44.13\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"354.76,-42.23 346.49,-35.6 348.82,-45.93 354.76,-42.23\"/>\n",
       "<text text-anchor=\"middle\" x=\"380.65\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\">False</text>\n",
       "</g>\n",
       "<!-- 7&#45;&gt;6 -->\n",
       "<g id=\"edge4\" class=\"edge\">\n",
       "<title>7&#45;&gt;6</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M399.87,-86.8C407.51,-74.54 417.91,-57.87 426.48,-44.13\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"429.48,-45.93 431.8,-35.6 423.54,-42.23 429.48,-45.93\"/>\n",
       "<text text-anchor=\"middle\" x=\"432.15\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\">True</text>\n",
       "</g>\n",
       "<!-- 8 -->\n",
       "<g id=\"node9\" class=\"node\">\n",
       "<title>8</title>\n",
       "<path fill=\"yellow\" stroke=\"black\" d=\"M336.15,-210C336.15,-210 234.15,-210 234.15,-210 228.15,-210 222.15,-204 222.15,-198 222.15,-198 222.15,-186 222.15,-186 222.15,-180 228.15,-174 234.15,-174 234.15,-174 336.15,-174 336.15,-174 342.15,-174 348.15,-180 348.15,-186 348.15,-186 348.15,-198 348.15,-198 348.15,-204 342.15,-210 336.15,-210\"/>\n",
       "<text text-anchor=\"middle\" x=\"285.15\" y=\"-188.3\" font-family=\"Times,serif\" font-size=\"14.00\">petal_width &gt; 1.55?</text>\n",
       "</g>\n",
       "<!-- 8&#45;&gt;4 -->\n",
       "<g id=\"edge5\" class=\"edge\">\n",
       "<title>8&#45;&gt;4</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M280.9,-173.8C278.02,-162.16 274.16,-146.55 270.87,-133.24\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"274.19,-132.04 268.39,-123.18 267.39,-133.72 274.19,-132.04\"/>\n",
       "<text text-anchor=\"middle\" x=\"290.65\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\">False</text>\n",
       "</g>\n",
       "<!-- 8&#45;&gt;7 -->\n",
       "<g id=\"edge6\" class=\"edge\">\n",
       "<title>8&#45;&gt;7</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M306.19,-173.8C321.7,-161.13 342.98,-143.73 360.1,-129.74\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"362.6,-132.21 368.13,-123.18 358.17,-126.79 362.6,-132.21\"/>\n",
       "<text text-anchor=\"middle\" x=\"357.15\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\">True</text>\n",
       "</g>\n",
       "<!-- 9 -->\n",
       "<g id=\"node10\" class=\"node\">\n",
       "<title>9</title>\n",
       "<path fill=\"yellow\" stroke=\"black\" d=\"M265.65,-297C265.65,-297 160.65,-297 160.65,-297 154.65,-297 148.65,-291 148.65,-285 148.65,-285 148.65,-273 148.65,-273 148.65,-267 154.65,-261 160.65,-261 160.65,-261 265.65,-261 265.65,-261 271.65,-261 277.65,-267 277.65,-273 277.65,-273 277.65,-285 277.65,-285 277.65,-291 271.65,-297 265.65,-297\"/>\n",
       "<text text-anchor=\"middle\" x=\"213.15\" y=\"-275.3\" font-family=\"Times,serif\" font-size=\"14.00\">petal_length &gt; 4.95?</text>\n",
       "</g>\n",
       "<!-- 9&#45;&gt;3 -->\n",
       "<g id=\"edge7\" class=\"edge\">\n",
       "<title>9&#45;&gt;3</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M198.58,-260.8C188.13,-248.47 173.91,-231.68 162.23,-217.89\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"164.83,-215.54 155.69,-210.18 159.49,-220.07 164.83,-215.54\"/>\n",
       "<text text-anchor=\"middle\" x=\"196.65\" y=\"-231.8\" font-family=\"Times,serif\" font-size=\"14.00\">False</text>\n",
       "</g>\n",
       "<!-- 9&#45;&gt;8 -->\n",
       "<g id=\"edge8\" class=\"edge\">\n",
       "<title>9&#45;&gt;8</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M227.72,-260.8C238.16,-248.47 252.38,-231.68 264.06,-217.89\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"266.8,-220.07 270.6,-210.18 261.46,-215.54 266.8,-220.07\"/>\n",
       "<text text-anchor=\"middle\" x=\"267.15\" y=\"-231.8\" font-family=\"Times,serif\" font-size=\"14.00\">True</text>\n",
       "</g>\n",
       "<!-- 10 -->\n",
       "<g id=\"node11\" class=\"node\">\n",
       "<title>10</title>\n",
       "<ellipse fill=\"lightgrey\" stroke=\"black\" cx=\"338.15\" cy=\"-279\" rx=\"42.49\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"338.15\" y=\"-275.3\" font-family=\"Times,serif\" font-size=\"14.00\">virginica</text>\n",
       "</g>\n",
       "<!-- 11 -->\n",
       "<g id=\"node12\" class=\"node\">\n",
       "<title>11</title>\n",
       "<path fill=\"yellow\" stroke=\"black\" d=\"M345.15,-384C345.15,-384 243.15,-384 243.15,-384 237.15,-384 231.15,-378 231.15,-372 231.15,-372 231.15,-360 231.15,-360 231.15,-354 237.15,-348 243.15,-348 243.15,-348 345.15,-348 345.15,-348 351.15,-348 357.15,-354 357.15,-360 357.15,-360 357.15,-372 357.15,-372 357.15,-378 351.15,-384 345.15,-384\"/>\n",
       "<text text-anchor=\"middle\" x=\"294.15\" y=\"-362.3\" font-family=\"Times,serif\" font-size=\"14.00\">petal_width &gt; 1.75?</text>\n",
       "</g>\n",
       "<!-- 11&#45;&gt;9 -->\n",
       "<g id=\"edge9\" class=\"edge\">\n",
       "<title>11&#45;&gt;9</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M277.75,-347.8C265.9,-335.36 249.7,-318.36 236.5,-304.5\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"238.95,-302 229.51,-297.18 233.88,-306.83 238.95,-302\"/>\n",
       "<text text-anchor=\"middle\" x=\"273.65\" y=\"-318.8\" font-family=\"Times,serif\" font-size=\"14.00\">False</text>\n",
       "</g>\n",
       "<!-- 11&#45;&gt;10 -->\n",
       "<g id=\"edge10\" class=\"edge\">\n",
       "<title>11&#45;&gt;10</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M303.05,-347.8C309.29,-335.74 317.75,-319.4 324.8,-305.79\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"327.92,-307.38 329.41,-296.89 321.7,-304.16 327.92,-307.38\"/>\n",
       "<text text-anchor=\"middle\" x=\"332.15\" y=\"-318.8\" font-family=\"Times,serif\" font-size=\"14.00\">True</text>\n",
       "</g>\n",
       "<!-- 12 -->\n",
       "<g id=\"node13\" class=\"node\">\n",
       "<title>12</title>\n",
       "<path fill=\"yellow\" stroke=\"black\" d=\"M254.65,-471C254.65,-471 159.65,-471 159.65,-471 153.65,-471 147.65,-465 147.65,-459 147.65,-459 147.65,-447 147.65,-447 147.65,-441 153.65,-435 159.65,-435 159.65,-435 254.65,-435 254.65,-435 260.65,-435 266.65,-441 266.65,-447 266.65,-447 266.65,-459 266.65,-459 266.65,-465 260.65,-471 254.65,-471\"/>\n",
       "<text text-anchor=\"middle\" x=\"207.15\" y=\"-449.3\" font-family=\"Times,serif\" font-size=\"14.00\">petal_width &gt; 0.8?</text>\n",
       "</g>\n",
       "<!-- 12&#45;&gt;0 -->\n",
       "<g id=\"edge11\" class=\"edge\">\n",
       "<title>12&#45;&gt;0</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M201.88,-434.8C198.27,-422.97 193.39,-407.03 189.28,-393.58\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"192.58,-392.42 186.31,-383.89 185.89,-394.47 192.58,-392.42\"/>\n",
       "<text text-anchor=\"middle\" x=\"210.65\" y=\"-405.8\" font-family=\"Times,serif\" font-size=\"14.00\">False</text>\n",
       "</g>\n",
       "<!-- 12&#45;&gt;11 -->\n",
       "<g id=\"edge12\" class=\"edge\">\n",
       "<title>12&#45;&gt;11</title>\n",
       "<path fill=\"none\" stroke=\"black\" d=\"M224.75,-434.8C237.49,-422.36 254.88,-405.36 269.06,-391.5\"/>\n",
       "<polygon fill=\"black\" stroke=\"black\" points=\"271.86,-393.67 276.57,-384.18 266.97,-388.66 271.86,-393.67\"/>\n",
       "<text text-anchor=\"middle\" x=\"269.15\" y=\"-405.8\" font-family=\"Times,serif\" font-size=\"14.00\">True</text>\n",
       "</g>\n",
       "</g>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<graphviz.graphs.Digraph at 0x7f3354d3cf90>"
      ]
     },
     "execution_count": 29,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cls = TreeClassifier(max_depth=6, criterion='gini')\n",
    "cls.fit(X2, Y)\n",
    "cls.draw_tree()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 500x500 with 1 Axes>"
      ]
     },
     "metadata": {
      "image/png": {
       "height": 446,
       "width": 456
      }
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "plot_boundary(cls, X2, Y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


NameError: name 'null' is not defined